In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from sklearn.linear_model import LinearRegression   #引入多元线性回归算法模块进行相应的训练
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1.出于对网上数据的不信任，请你帮他快速检验附件一中30只股票的数据是否存在异常值（例如股价涨跌幅超过了10%限制等），缺失值，并给出解决方案。注意，禁止通过查询该只股票的真实数据来处理异常值和缺失值。

In [ ]:
stocks=pd.read_csv('/kaggle/input/130886/Appendix 1 Quotes of 30 stocks.csv',usecols=[1,2,3,4,5,6,7,8,9])
stocks.head(2)

In [ ]:
stocks['time']=pd.to_datetime(stocks['time'],format="%Y/%m/%d %H:%M")

In [ ]:
stocks.isnull().sum()#查看缺失值

In [ ]:
stocks.drop('open_interest',inplace=True,axis=1)

In [ ]:
stocks

In [ ]:
#找出含有缺失值的行
stocks[stocks[['time', 'code', 'open', 'high', 'low', 'close', 'volume', 'amount']].isnull().any(axis=1)]

In [ ]:
#股票'szse.000636'最后一天的数据colse数据缺失,采用前两天的均值填充
temp=stocks[stocks['code']=='szse.000636']
temp[temp['time']<datetime.datetime.strptime('2019/3/31 15:00',"%Y/%m/%d %H:%M")].tail()

In [ ]:
stocks.loc[8803,'close']=(257.818805+257.649187)/2
stocks.loc[8803]

In [ ]:
#股票'sse.600325'一天的数据open数据缺失,采用前一天的clse数据填充
temp=stocks[stocks['code']=='sse.600325']
temp[temp['time']<datetime.datetime.strptime('2017/9/13 15:00',"%Y/%m/%d %H:%M")].tail(5)

In [ ]:
stocks.loc[22597,'open']=61.427646
stocks.loc[22597]

In [ ]:
stocks['code'].unique()

In [ ]:
stocks[stocks['time']==stocks.loc[33067,'time']]['code'].unique()

In [ ]:
#可以确定缺失的code是'sse.601318'
stocks.loc[33067,'code']='sse.601318'
stocks.loc[33067]

In [ ]:
#股票'sse.600332'数据缺失,采用前后两天的均值填充
temp=stocks[stocks['code']=='sse.600332']
temp[temp['time']<datetime.datetime.strptime('2011/5/25 15:00',"%Y/%m/%d %H:%M")].tail()

In [ ]:
stocks.loc[63033,'volume']=(33625.0+28512.0)/2
stocks.loc[63033]

In [ ]:
#四个缺失值均已经填充
stocks[stocks[['time', 'code', 'open', 'high', 'low', 'close', 'volume', 'amount']].isnull().any(axis=1)]

In [ ]:
stocks['yesterdayClose']=stocks['close']

In [ ]:
def findYesterdayClose(df):
    df=pd.DataFrame(df)
    df.sort_values("time",inplace=True)
    df.reset_index(inplace=True,drop=True)
    for i in range(1,df.shape[0]):
        df.loc[i,'yesterdayClose']=df.loc[i-1,'close']
    return df

In [ ]:
#涨幅=(现价-上一个交易日收盘价)/上一个交易日收盘价*100%
temp=stocks.groupby(by=['code'],axis=0).apply(findYesterdayClose)
temp.reset_index(drop=True,inplace=True)
temp.head(3)

In [ ]:
#涨幅=(现价-上一个交易日收盘价)/上一个交易日收盘价*100%
#此时，open_interest是上一个交易日收盘价
temp['up']=(temp['high']-temp['yesterdayClose'])/temp['yesterdayClose']
temp['down']=(temp['low']-temp['yesterdayClose'])/temp['yesterdayClose']
temp.head(2)

In [ ]:
#以下数据涨幅超过了10%
temp[abs(temp['up']*10)>1.000000001]

In [ ]:
#以下数据跌超过了10%
temp[abs(temp['down']*10)>1.000000001]

In [ ]:
#异常数据修正
temp['high']=(abs(temp['up'])>0.1)*temp['yesterdayClose']*1.1+(abs(temp['up'])<=0.1)*temp['high']
temp['low']=(abs(temp['down'])>0.1)*temp['yesterdayClose']*0.9+(abs(temp['down'])<=0.1)*temp['low']

In [ ]:
stocks[['time', 'code', 'open', 'high', 'low', 'close', 'volume', 'amount']]=temp[['time', 'code', 'open', 'high', 'low', 'close', 'volume', 'amount']]

In [ ]:
stocks.drop(['yesterdayClose'],axis=1,inplace=True)

In [ ]:
stocks.head(3)

# 2.大湾区指数是由这30支股票线性组合而成，请你利用附件一和附件二的数据解出大湾区指数中30支股票的权重，并回答从2011年至今指数的权重构成是否发生改变。

In [ ]:
stockIndex=pd.read_csv('../input/130886/Appendix 2 Greater Bay Area Index Market.csv',usecols=[1,2,3,4,5,6,7,8,9])
stockIndex.head(2)

In [ ]:
stockIndex['time']=pd.to_datetime(stockIndex['time'],format="%Y/%m/%d %H:%M")

In [ ]:
stockIndex['code'].unique()

In [ ]:
stockIndex.shape

In [ ]:
stockIndex.isnull().sum()

In [ ]:
stockIndex.drop('open_interest',inplace=True,axis=1)

In [ ]:
stocks.shape

In [ ]:
stocks.isnull().sum()

In [ ]:
temp=stocks.append(stockIndex)

In [ ]:
temp.head()

In [ ]:
col=['time', 'year','sse.600048', 'sse.600183', 'sse.600323', 'sse.600325', 'sse.600332',
       'sse.600383', 'sse.600872', 'sse.601318', 'szse.000028', 'szse.000049',
       'szse.000069', 'szse.000513', 'szse.000636', 'szse.000651',
       'szse.000921', 'szse.001914', 'szse.002027', 'szse.002035',
       'szse.002060', 'szse.002138', 'szse.002152', 'szse.002233',
       'szse.002303', 'szse.002352', 'szse.002449', 'szse.002461',
       'szse.002475', 'szse.002511', 'szse.300014', 'szse.300115',
       'szse.399999']
def createFeature(df,columns):
    f=df.pivot(index='time', columns='code', values=columns)
    f['time']=f.index
    f.reset_index(drop=True,inplace=True)
    f['year']=f['time'].dt.year#获取年分
    f.dropna(axis=0, inplace=True)#有些股票停牌了，要去掉
    return f[col]

In [ ]:
fitColumn='open'
#这个产生fitColumn的数据集，每一行包含time,年，30支股票和打完去指数,值是fitColum
#fitColumn可以换成'open', 'high', 'low', 'close', 'volume', 'amount'
Featue=createFeature(temp,fitColumn)
Featue.head(2)

In [ ]:
xCol=['sse.600048', 'sse.600183', 'sse.600323', 'sse.600325', 'sse.600332',
       'sse.600383', 'sse.600872', 'sse.601318', 'szse.000028', 'szse.000049',
       'szse.000069', 'szse.000513', 'szse.000636', 'szse.000651',
       'szse.000921', 'szse.001914', 'szse.002027', 'szse.002035',
       'szse.002060', 'szse.002138', 'szse.002152', 'szse.002233',
       'szse.002303', 'szse.002352', 'szse.002449', 'szse.002461',
       'szse.002475', 'szse.002511', 'szse.300014', 'szse.300115']
#30支股票
yCol=['szse.399999']
#大湾区指数

In [ ]:
model=LinearRegression()
x=Featue[xCol]
y=Featue[yCol]
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=666)#划分训练集测试集
model.fit(x_train,y_train)
print(model.coef_)               #输出多元线性回归的各项系数
print(model.intercept_)          #输出多元线性回归的常数项的值
y_predict=model.predict(x_test)
print(mean_squared_error(y_test,y_predict))
print(mean_absolute_error(y_test,y_predict))
print(r2_score(y_test,y_predict))
print(model.score(x_test,y_test))
print(model.coef_)               #输出多元回归算法的各个特征的系数矩阵
print(np.argsort(model.coef_))  #输出多元线性回归算法各个特征的系数排序，可以知道各个特征的影响度
#输出各个特征按照影响系数从小到大的顺序
[xCol[int(i)] for i in np.argsort(model.coef_)[0]]

In [ ]:
#检验每年系数是否一致
for year in Featue.year.unique():
    print('year = ',year)
    model=LinearRegression()
    x=Featue[Featue['year']==year][xCol]
    y=Featue[Featue['year']==year][yCol]
    x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=666)#划分训练集测试集
    model.fit(x_train,y_train)
    print(model.coef_)               #输出多元线性回归的各项系数
    print(model.intercept_)          #输出多元线性回归的常数项的值

可以看出’open'在2015，2016年发生了变化，其它可修改fitColumn，fitColumn可以为'open', 'high', 'low', 'close', 'volume', 'amount'

# 3.附件一中只显示了股票的代码，小明表示不太习惯，请你在读入附件一后得到的DataFrame中，在“股票代码”这一列后新增一列“股票名称”。注意：股票名称允许使用真实数据搜寻，也可建立映射表实现。

In [ ]:

name={}
name['600048']='保利地产'
name['600183']='生益科技'
name['600323']='瀚蓝环境'
name['600325']='华发股份'
name['600332']='白云山'
name['600383']='金地集团'
name['600872']='中炬高新'
name['000028']='国药一致'
name['000049']='德赛电池'
name['000069']='华侨城Ａ'

name['002303']='美盈森'
name['002352']='顺丰控股'
name['002449']='国星光电'
name['002461']='珠江啤酒'
name['002475']='立讯精密'
name['002511']='中顺洁柔'
name['300115']='长盈精密'


name['000513']='丽珠集团'
name['000636']='风华高科'
name['000651']='格力电器'
name['000921']='海信家电'
name['001914']='招商积余'
name['002027']='分众传媒'
name['002060']='粤水电'
name['002138']='顺络电子'
name['002152']='广电运通'
name['002233']='塔牌集团'
name['601318']='中国平安'
name['002035']='华帝股份'
name['300014']='亿纬锂能'

In [ ]:
len(list(name.keys()))

In [ ]:
len(set(name.values()))

In [ ]:
stocks['股票名称']=stocks['code'].apply(lambda x:name[x.split('.')[1]])
stocks

# 4.小明认为，那些曾经停牌过的股票并不安全，尤其是停牌天数很多的股票。请你撰写一个函数，该函数能够实现查询并返回任意股票的停牌天数及停牌的具体时间。

In [ ]:
infor=stocks.pivot(index='time', columns='code', values='open')

In [ ]:
infor['time']=infor.index
infor.reset_index(drop=True,inplace=True)

In [ ]:
stocks.isnull().sum()

In [ ]:
def findStop(code,df):
    infor=df.pivot(index='time', columns='code', values='open')
    infor['time']=infor.index
    infor.reset_index(drop=True,inplace=True)
    dayNum=infor[code].isnull().sum()#停牌天数
    days=infor[infor[code].isnull()]['time'].tolist()#停牌时间
    return dayNum,days

In [ ]:
for code in stocks['code'].unique():
    dayNum,days=findStop(code,stocks)
    print('股票：',code,name[code.split('.')[1]],'停牌 ',dayNum,' 天')
    
    
    #输出停牌天
    print(days)

# 5. 小明想要基于此来建立一个指数增强策略，该策略目的是对指数进行跟踪，通过选股、择时来获得超额收益（在尽可能实现跟踪指数的情况下获取增强收益）。

In [ ]:
t5=stocks.copy(deep=True)

In [ ]:
t5['weekday']=t5['time'].dt.weekday#获取周几的信息
t5.head(3)

今天是周一你开始执行第一次执行策略。假如你计算得到了上周最后一天涨幅最大的10只股票A,B,C,D,... 那么你就在周一收盘时买入。直到下周一的收盘时，这是你第二次执行策略，你计算得到策略第一周最后一天涨幅最大的10只股票，如果你已经持有，则继续持有，否则卖出结算收益。

In [ ]:
#涨幅=(现价-上一个交易日收盘价)/上一个交易日收盘价*100%
t5FindBest10Stocks=t5.groupby(by=['code'],axis=0).apply(findYesterdayClose)
t5FindBest10Stocks.reset_index(drop=True,inplace=True)
t5FindBest10Stocks.head(3)

In [ ]:
#如果是第一天就要用这一天的close代替上一天的close
t5FindBest10Stocks['yesterdayClose']=t5FindBest10Stocks['yesterdayClose'].isnull()*t5FindBest10Stocks['close']+t5FindBest10Stocks['yesterdayClose'].fillna(0)

In [ ]:
t5FindBest10Stocks.head()

In [ ]:
t5FindBest10Stocks['up']=(t5FindBest10Stocks['close']-t5FindBest10Stocks['yesterdayClose'])/t5FindBest10Stocks['yesterdayClose']

In [ ]:
t5FindBest10Stocks=t5FindBest10Stocks[t5FindBest10Stocks['weekday']==0].pivot(index='time', columns='code', values='up')
codes=t5FindBest10Stocks.columns.tolist()

In [ ]:
t5FindBest10Stocks.head()[xCol[:3]]

In [ ]:
t5FindBest10Stocks.head()[xCol[:3]].apply(np.argsort,axis=1)

In [ ]:
#找到涨幅最大的10只股票代码
def selctCols(df):
    cols=[]
    for i in range(df.shape[0]):
        if df[i]>=20:
            cols.append(codes[i])
        if df[i]==-1:
            cols.append(codes[i])
    return cols
        

In [ ]:
#每周末涨幅最大的10只股票代码
pd.DataFrame(t5FindBest10Stocks.head().apply(np.argsort,axis=1).apply(selctCols,axis=1),columns=['Best10Stocks'])

剩下的不会了

# 6.小明想实现给定一段时间和金额，返回这段时间内如何交易股票能够获取最大收益，并给出最大收益（采用T+1，且只在开盘或收盘交易）。注意，此时你相当于上帝，因为你知晓当日和未来的股票涨幅，所以返回最大收益是可行的。

In [ ]:
t6=stocks.copy(deep=True)

## 我们是以上帝视角在买股票，因此我们只在第二天涨幅最大的股票的涨幅大于0时买股票，此时我们在前一天的low时买入，在第二天high时卖出
## 前一天把low设置为委托价格，到达委托价格尽最大努力买入，第二天在把high的价格设置为委托的价格，到达委托价格就全部卖出
## 我们不知道每支股票一股多少钱，我们假设一股20

In [ ]:
#找到上一个交易日的开盘价格和收盘价格
def findYesterdayLow(df):
    df=pd.DataFrame(df)
    df.sort_values("time",inplace=True)
    df.reset_index(inplace=True,drop=True)
    df.loc[0,'yesterdayLow']=df.loc[0,'low']
    for i in range(1,df.shape[0]):
        df.loc[i,'yesterdayLow']=df.loc[i-1,'low']
    return df

In [ ]:
t6['yesterdayLow']=t6['low']

In [ ]:
#涨幅=(现价-上一个交易日收盘价)/上一个交易日收盘价*100%
temp=t6.groupby(by=['code'],axis=0).apply(findYesterdayLow)
temp.reset_index(drop=True,inplace=True)
temp.head(3)

In [ ]:
temp['up']=(temp['high']-temp['yesterdayLow'])/temp['yesterdayLow']

In [ ]:
gold=temp.pivot(index='time', columns='code', values='up')
gold.head()

In [ ]:
gold.fillna(-1000,inplace=True)

In [ ]:
gold['time']=gold.index
gold.reset_index(drop=True,inplace=True)

In [ ]:
gold.fillna(-1000,inplace=True)

In [ ]:
x=gold.columns[:-1]

## 我们是以上帝视角在买股票，因此我们只在第二天涨幅最大的股票的涨幅大于0时买股票，此时我们在前一天的low时买入，在第二天high时卖出
## 前一天把low设置为委托价格，到达委托价格尽最大努力买入，第二天在把high的价格设置为委托的价格，到达委托价格就全部卖出
## 我们不知道每支股票一股多少钱，我们假设一股20

In [ ]:
start='2017/9/13 15:00'
end='2017/10/13 15:00'
def buyStocks(start,end,money):
    temp=gold[gold['time']>=datetime.datetime.strptime(start,"%Y/%m/%d %H:%M")]
    temp=temp[temp.time<=datetime.datetime.strptime(end,"%Y/%m/%d %H:%M")]
    temp.reset_index(drop=True,inplace=True)
    up=temp.loc[1:,][x].apply(max,axis=1).tolist()
    Sum=money
    for i in up:
        if i>0:
            Sum=Sum+(20*100)*(Sum//(20*100))*i
    profit=Sum-money
    return profit

In [ ]:
start='2017/9/13 15:00'
end='2017/10/13 15:00'
buyStocks(start,end,money=10000)